In [1]:
import nltk
# nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd
import pickle
import random
import json

Importing our dataset

In [2]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

Bulding our list of words, classes and words to tags mapping(documents)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?',',','.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmatize and lower each word
lemma_function = WordNetLemmatizer()
words = [lemma_function.lemmatize(w.lower()) for w in words if w not in ignore_words]

# remove duplicates
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))

# words = all words, vocabulary
print (len(words), "lemmatized words", words)
# documents = combination between patterns and intents
print (len(documents), "documents", documents)
# classes = intents
print (len(classes), "classes", classes)

85 lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hi', 'history', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']
45 documents [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later

Creating training data from dataset

In [4]:
training = []
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    bag = []
    pattern_words = doc[0]
    # lemmatize each word
    pattern_words = [lemma_function.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

/var/folders/fj/l84qvx552y37ht23spc9y0140000gn/T/ipykernel_31955/1985779988.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Building the Neural Network Model

In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

2021-11-18 02:26:43.737633: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Training the model on training data

In [7]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
9/9 [==============================] - 1s 1ms/step - loss: 2.2321 - accuracy: 0.0889
Epoch 2/200
9/9 [==============================] - 0s 1ms/step - loss: 2.1841 - accuracy: 0.2000
Epoch 3/200
9/9 [==============================] - 0s 1ms/step - loss: 2.0309 - accuracy: 0.3111
Epoch 4/200
9/9 [==============================] - 0s 1ms/step - loss: 1.9893 - accuracy: 0.3778
Epoch 5/200
9/9 [==============================] - 0s 1ms/step - loss: 1.8460 - accuracy: 0.3556
Epoch 6/200
9/9 [==============================] - 0s 2ms/step - loss: 1.6830 - accuracy: 0.6000
Epoch 7/200
9/9 [==============================] - 0s 1ms/step - loss: 1.4883 - accuracy: 0.5556
Epoch 8/200
9/9 [==============================] - 0s 2ms/step - loss: 1.4379 - accuracy: 0.6667
Epoch 9/200
9/9 [==============================] - 0s 1ms/step - loss: 1.4547 - accuracy: 0.5333
Epoch 10/200
9/9 [==============================] - 0s 2ms/step - loss: 1.2884 - accuracy: 0.5778
Epoch 11/200
9/9 [===========

Functions to preprocess input

In [8]:
def get_words_from_input_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemma_function.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def get_input_as_bag_of_words(sentence, words, show_details=True):
    sentence_words = get_words_from_input_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

Taking input message and converting to bag of words

In [9]:
input_sentence = "Which is the nearest pharmacy to me?"

In [10]:
bag_of_words_input = get_input_as_bag_of_words(input_sentence, words)
print ("Input sentence as bag of words - ",bag_of_words_input)
print ("Possible output classes - ",classes)

found in bag: which
found in bag: is
found in bag: pharmacy
found in bag: to
found in bag: me
Input sentence as bag of words -  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 0]
Possible output classes -  ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


Predict output class using our model

In [11]:
inputvar = pd.DataFrame([bag_of_words_input], dtype=float, index=['input'])
model_output = model.predict(inputvar)
print("model output - ", model_output)

model output -  [[1.5237112e-05 2.5038205e-05 7.9430347e-06 9.7650227e-06 1.1124955e-06
  1.7185192e-06 5.9266840e-06 9.9992216e-01 1.1086969e-05]]


In [12]:
print("Input sentence - ", input_sentence)
print("Output class for input - ", classes[model_output.argmax()])

Input sentence -  Which is the nearest pharmacy to me?
Output class for input -  pharmacy_search


Save model and data structures to pickle files

In [13]:
pickle.dump(model, open("chatbot-model.pkl", "wb"))

2021-11-18 02:26:57.737643: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://1331870f-7afe-4d9f-a2b0-4db93f0653b3/assets


In [14]:
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "chatbot-data.pkl", "wb" ) )